In [41]:
# simple linear neural network

import random
import torch

class Adder(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Adder, self).__init__()
        self.encoder = torch.nn.Linear(input_size * 2, hidden_size)
        self.decoder = torch.nn.Linear(hidden_size, 1)

    def forward(self, x1, x2):
        x1 = x1.reshape(-1, 1)
        x2 = x2.reshape(-1, 1)
        x = torch.cat([x1, x2], dim=1)
        h = torch.relu(self.encoder(x))
        y = self.decoder(h)
        return y

In [42]:
from torchsummary import summary

# Generate a dataset of input-output pairs
dataset_size = 1000
input_size = 1
hidden_size = 10
dataset = []
for i in range(dataset_size):
    x1 = random.uniform(-10, 10)
    x2 = random.uniform(-10, 10)
    y = x1 + x2
    dataset.append((torch.tensor(x1), torch.tensor(x2), torch.tensor(y)))

# Split the dataset into training and validation sets
train_size = int(0.8 * dataset_size)
train_dataset = dataset[:train_size]
val_dataset = dataset[train_size:]

# Create an instance of the Adder network
net = Adder(input_size, hidden_size)
summary(net, input_size=[(1,), (1,)])

# Define the loss function and the optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              30
            Linear-2                    [-1, 1]              11
Total params: 41
Trainable params: 41
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [ ]:
# Train the network on the training set
import matplotlib.pyplot as plt

num_epochs = 1000
for epoch in range(num_epochs):
    running_loss = 0.0
    for x1, x2, y in train_dataset:
        optimizer.zero_grad()
        y_pred = net(x1, x2)
        loss = criterion(y_pred, y.reshape(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_dataset)

    # Evaluate the network on the validation set
    running_loss = 0.0
    for x1, x2, y in val_dataset:
        y_pred = net(x1, x2)
        loss = criterion(y_pred, y.reshape(-1, 1))
        running_loss += loss.item()
    val_loss = running_loss / len(val_dataset)

    # Print the loss for this epoch
    if epoch % 100 == 99:
        print(f"Epoch {epoch+1}: train_loss={train_loss:.6f}, val_loss={val_loss:.6f}")

In [51]:
# Test the network on a new input
x1 = float(input("Enter the first number: "))
x2 = float(input("Enter the second number: "))
final_pred = net(torch.tensor(x1), torch.tensor(x2))
print("The sum of", x1, "and", x2, "is:", final_pred.item())

The sum of 60.0 and 70.0 is: 129.9859161376953
